## Boiler plate code

In [1]:
from os import environ
from pyspark.sql import SparkSession, Row

In [2]:
spark = SparkSession.builder.appName("friends_df").getOrCreate()

In [3]:
def mapper(line):
    fields = line.split(",")
    return Row(ID=int(fields[0]), name=str(fields[1].encode("utf-8")), \
                  age = int(fields[2]), num_friends = int(fields[3]))

### Creating RDD using spark.sparkContext

In [4]:
file_path = "file:///"+environ['DATA_LAKE']
lines = spark.sparkContext.textFile(file_path+"friends.csv")

In [5]:
people = lines.map(mapper)

### Infer schema and register DataFrame as a table

In [6]:
people_df = spark.createDataFrame(people)

In [7]:
people_df.createOrReplaceTempView("people")

### write queries like normal table

In [8]:
teenagers = spark.sql("select * from people where age >=13 and age <=19").head(1)
teenagers

[Row(ID=21, name="b'Miles'", age=19, num_friends=268)]

In [9]:
people_df.groupby("age").count().orderBy("count").limit(5).show()

+---+-----+
|age|count|
+---+-----+
| 63|    4|
| 20|    5|
| 50|    5|
| 65|    5|
| 24|    5|
+---+-----+



In [10]:
spark.stop()